### 必要なライブラリのインポート

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta

### アプリIDをWebページから抽出

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0"
}

base_url = "https://game-i.daa.jp/?アプリサービス終了分析"
app_ids = set()

# 1ページ目（例外）
print(f"取得中: {base_url}")
res = requests.get(base_url, headers=headers, timeout=10)
ids = re.findall(r'2F(\d{10})', res.text)
app_ids.update(ids)
time.sleep(1)

# 2ページ目以降（6か月ずつ遡る）
current = datetime.strptime("2025-02", "%Y-%m")
limit = datetime.strptime("2018-01", "%Y-%m")

while current >= limit:
    ym_str = current.strftime("%Y-%m")
    url = f"https://game-i.daa.jp/?アプリサービス終了分析&Ym={ym_str}"
    print(f"取得中: {url}")
    try:
        res = requests.get(url, headers=headers, timeout=10)
        res.raise_for_status()
    except Exception as e:
        print(f"エラー: {e}")
        break

    found_ids = re.findall(r'2F(\d{10})', res.text)
    if not found_ids:
        print(f"IDなし: {ym_str}")
        break

    app_ids.update(found_ids)
    time.sleep(1)
    current -= relativedelta(months=6)

# 結果出力
app_ids = sorted(list(app_ids))
print(f"\n 抽出完了: {len(app_ids)} 件")


取得中: https://game-i.daa.jp/?アプリサービス終了分析
取得中: https://game-i.daa.jp/?アプリサービス終了分析&Ym=2025-02
取得中: https://game-i.daa.jp/?アプリサービス終了分析&Ym=2024-08
取得中: https://game-i.daa.jp/?アプリサービス終了分析&Ym=2024-02
取得中: https://game-i.daa.jp/?アプリサービス終了分析&Ym=2023-08
取得中: https://game-i.daa.jp/?アプリサービス終了分析&Ym=2023-02
取得中: https://game-i.daa.jp/?アプリサービス終了分析&Ym=2022-08
取得中: https://game-i.daa.jp/?アプリサービス終了分析&Ym=2022-02
取得中: https://game-i.daa.jp/?アプリサービス終了分析&Ym=2021-08
取得中: https://game-i.daa.jp/?アプリサービス終了分析&Ym=2021-02
取得中: https://game-i.daa.jp/?アプリサービス終了分析&Ym=2020-08
取得中: https://game-i.daa.jp/?アプリサービス終了分析&Ym=2020-02
取得中: https://game-i.daa.jp/?アプリサービス終了分析&Ym=2019-08
取得中: https://game-i.daa.jp/?アプリサービス終了分析&Ym=2019-02
取得中: https://game-i.daa.jp/?アプリサービス終了分析&Ym=2018-08
取得中: https://game-i.daa.jp/?アプリサービス終了分析&Ym=2018-02
IDなし: 2018-02

 抽出完了: 393 件


### CSVに保存

In [3]:
pd.DataFrame({"app_id": app_ids}).to_csv("extracted_app_ids.csv", index=False)

### アプリ情報を抽出

In [4]:
app_ids_df = pd.read_csv("extracted_app_ids.csv")
app_ids = app_ids_df['app_id'].astype(str).tolist()

headers = {
    "User-Agent": "Mozilla/5.0"
}

results = []

for app_id in app_ids:
    url = f"https://game-i.daa.jp/?APP%2F{app_id}"
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except Exception as e:
        print(f"エラー: {app_id} - {e}")
        continue

    soup = BeautifulSoup(response.text, "html.parser")

    # タイトル抽出（「アプリ/」と「【売上＆人気アプリランキング】」を除去）
    title_tag = soup.find("title")
    if not title_tag:
        print(f"スキップ（タイトルなし）: {app_id}")
        continue

    title_raw = title_tag.text.strip()
    match = re.search(r"アプリ/(.+?)【売上＆人気アプリランキング】", title_raw)
    if match:
        title = match.group(1).strip()
    else:
        print(f"スキップ（形式外タイトル）: {app_id}")
        continue

    # 関連ジャンル
    genre_tag = soup.find("td", string=re.compile("関連ジャンル"))
    genre = genre_tag.find_next_sibling("td").text.strip().replace("\n", " ").replace(",", " ") if genre_tag else ""

    # サービス終了日（<strong>タグ内にある場合に対応）
    end_service = ""
    for td in soup.find_all("td", class_="style_td"):
        if "サービス終了" in td.get_text():
            strong = td.find("strong")
            if strong:
                end_service = strong.get_text(strip=True)
                break

    # 年月と売上予測（2018〜2025）
    for year in range(2018, 2026):
        panel = soup.find("div", id=f"panel{year}")
        if not panel:
            continue
        rows = panel.find_all("tr", class_="customtbl")
        for row in rows[1:]:
            cols = row.find_all("td")
            if len(cols) < 2:
                continue
            ym_tag = cols[0].find("a") or cols[0].find("span")
            sales_tag = cols[1].find("span")
            if not ym_tag or not sales_tag:
                continue
            ym = ym_tag.text.strip()
            sales = sales_tag.text.strip()
            if not re.match(r"^\d{4}/\d{2}$", ym):
                continue
            if sales in ["-", "未", ""]:
                continue
            results.append({
                "タイトル": title,
                "年月": f'="{ym}"',
                "売上予測": sales,
                "関連ジャンル": genre,
                "サービス終了日": end_service
            })

    print(f"抽出完了: {title}")
    time.sleep(1)  # アクセス間隔

抽出完了: スーパーガンダムロワイヤル
抽出完了: 武器よさらば
抽出完了: スーパーロボット大戦X-Ω
抽出完了: FINAL FANTASY GRANDMASTERS
抽出完了: ドラゴンプロジェクト
抽出完了: 戦の海賊
抽出完了: 誰ガ為のアルケミスト
抽出完了: デッキヒーローズ-本格派戦略カードゲーム-
抽出完了: 美少女戦士セーラームーン　セーラームーンドロップス
抽出完了: モンスターハンター エクスプロア
抽出完了: 夢色キャスト
抽出完了: グリムノーツ
抽出完了: DAME×PRINCE -ダメ王子たちとのドタバタ恋愛ADV
抽出完了: かんぱにガールズ ファンタジーRPG
抽出完了: デジモンリンクス
抽出完了: BFBチャンピオンズ2.0
抽出完了: フェアリーテイル 極・魔法乱舞
抽出完了: 12オーディンズ - 王道RPG
抽出完了: ラストピリオド - 終わりなき螺旋の物語 -
抽出完了: 聖闘士星矢 ゾディアック ブレイブ
抽出完了: ブレイブファンタジア【まったり＆簡単操作の爽快RPG】
抽出完了: 新発売　おそ松さんのへそくりウォーズ　〜ニートの攻防〜
抽出完了: 野球つく！！
抽出完了: VALKYRIE ANATOMIA(ヴァルキリーアナトミア)
抽出完了: ONE PIECE サウザンドストーム
抽出完了: 仮面ライダー バトルラッシュ
抽出完了: 実況パワフルサッカー 【選手育成サッカーゲーム】
抽出完了: スマッシュ＆マジック
抽出完了: オルタナティブガールズ
抽出完了: クックと魔法のレシピ おかわり（育成ゲーム）
抽出完了: セブンズストーリー
抽出完了: HIDE AND FIRE（ハイドアンドファイア）
抽出完了: 8 beat Story ~アイドル×音楽ゲーム~
抽出完了: テイルズ オブ ザ レイズ
抽出完了: NARUTO-ナルト- 疾風伝　ナルティメットブレイジング
抽出完了: ディシディアファイナルファンタジー オペラオムニア
抽出完了: ディズニー マイリトルドール
抽出完了: ハイキュー!!ドンピシャマッチ!!
抽出完了: ドールズオーダー 【チーム対戦アクション】
抽出完了: THE NEW GATE（ザ・ニュー・ゲート）
抽出完了: スーパー戦隊レジェンドウォーズ
抽出完了: 三国ブレイズ

### 年月列をクリーンアップしてからCSV保存

In [ ]:
df = pd.DataFrame(results)

# 年月列のクリーンアップ処理
if "年月" in df.columns:
    df["年月"] = (
        df["年月"]
        .astype(str)
        .str.replace(r'^[=]*["]*', '', regex=True)  # 先頭の = や " を除去
        .str.replace(r'["]*$', '', regex=True)      # 末尾の " を除去
        .str.replace("/", "-")                      # スラッシュ → ハイフン
    )
df.to_csv("gamei_apps_sales_summary_utf8bom_ver2.csv", index=False, encoding="utf-8-sig")